<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/matrix_factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo Prático: Fatoração de Matrizes (Matrix Factorization)

A Fatoração de Matrizes é uma das técnicas mais poderosas e populares para sistemas de recomendação. É um método de **Filtragem Colaborativa baseada em modelo**.

A ideia principal é decompor a matriz de avaliações (geralmente esparsa) `R` em duas matrizes de dimensões menores, `P` e `Q`.

-   **R (m x n)**: Matriz de avaliações, com `m` usuários e `n` itens.
-   **P (m x k)**: Matriz de **fatores latentes dos usuários**. Cada linha é um vetor que representa o perfil de um usuário.
-   **Q (n x k)**: Matriz de **fatores latentes dos itens**. Cada linha é um vetor que representa o perfil de um item.

O objetivo é encontrar `P` e `Q` tal que o produto $P \times Q^T$ se aproxime da matriz original `R`. Os valores "descobertos" nessa nova matriz preenchida são as nossas previsões de notas.

$R \approx P \times Q^T$

Os **fatores latentes (k)** são características abstratas que o algoritmo aprende sozinho (ex: para filmes, pode ser "nível de comédia", "quantidade de ação", "drama", etc.).

Neste notebook, usaremos a biblioteca `surprise` para aplicar o **SVD (Singular Value Decomposition)**, um popular algoritmo de fatoração de matrizes.

In [ ]:
# A biblioteca 'surprise' é específica para sistemas de recomendação.
# Se não a tiver, descomente e execute a linha abaixo.
# !pip install scikit-surprise

import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

print("Bibliotecas importadas com sucesso!")

In [ ]:
# Usaremos o mesmo dataset do notebook anterior para manter a consistência.
data = {
    'Ana': {'Matrix': 5, 'Titanic': 3, 'O Poderoso Chefão': 4, 'Forrest Gump': None, 'Interestelar': 5},
    'Bruno': {'Matrix': 5, 'Titanic': 2, 'O Poderoso Chefão': None, 'Forrest Gump': 3, 'Interestelar': 4},
    'Carla': {'Matrix': 2, 'Titanic': 5, 'O Poderoso Chefão': 2, 'Forrest Gump': 5, 'Interestelar': 2},
    'Daniel': {'Matrix': None, 'Titanic': 4, 'O Poderoso Chefão': 5, 'Forrest Gump': 5, 'Interestelar': None},
    'Elisa': {'Matrix': 4, 'Titanic': None, 'O Poderoso Chefão': 5, 'Forrest Gump': 2, 'Interestelar': 5}
}
ratings_df = pd.DataFrame(data)

print("Matriz Original de Avaliações:")
print(ratings_df)

# A biblioteca 'surprise' precisa dos dados em um formato "longo": (usuário, item, avaliação).
# Vamos converter nosso DataFrame.
df_long = ratings_df.stack().reset_index()
df_long.columns = ['item', 'user', 'rating']

print("\n\nDados no formato 'longo' para o Surprise:")
print(df_long.head())

# Agora, carregamos os dados no formato que o Surprise entende.
# O Reader define a escala das notas (1 a 5).
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(df_long[['user', 'item', 'rating']], reader)

# Vamos usar o conjunto de dados completo para treinar e ver como ele preenche os vazios.
trainset = data_surprise.build_full_trainset()
print("\nDados carregados no Surprise com sucesso.")

In [ ]:
"""
### Passo 1: Treinar o Modelo

Instanciamos o SVD, definindo o número de fatores latentes (`n_factors`). Este é o `k` da nossa explicação. Um `k` pequeno ajuda a generalizar e evitar overfitting.
"""

# Usando k=2 fatores latentes para este exemplo
algo = SVD(n_factors=2, random_state=42)

# Treinando o algoritmo com nossos dados
algo.fit(trainset)

print("Modelo SVD treinado!")

In [ ]:
"""
### Passo 2: Prever Notas Faltantes

O principal objetivo da fatoração de matrizes é prever as notas que não existem. Vamos prever qual nota a `Ana` daria para `Forrest Gump`.
"""
user_to_predict = 'Ana'
item_to_predict = 'Forrest Gump'

# A nota original era NaN (não avaliado)
original_rating = ratings_df.loc[item_to_predict, user_to_predict]
print(f"Nota original de '{user_to_predict}' para '{item_to_predict}': {original_rating}")

# Fazendo a previsão
prediction = algo.predict(uid=user_to_predict, iid=item_to_predict)
predicted_rating = prediction.est

print(f"Nota prevista pelo SVD: {predicted_rating:.2f}")


In [ ]:
"""
### Passo 3: Visualizar a Matriz Reconstruída

Vamos usar nosso modelo treinado para preencher TODOS os valores, incluindo os que já existiam e os que estavam faltando, para ver a matriz completa.
"""
# Criando uma cópia da matriz original para preencher com as previsões
reconstructed_df = ratings_df.copy()

# Iterando por todos os usuários e itens para preencher a matriz
for user in ratings_df.columns:
    for item in ratings_df.index:
        reconstructed_df.loc[item, user] = algo.predict(uid=user, iid=item).est

print("Matriz Original (com valores NaN):")
display(ratings_df.style.highlight_null(null_color='lightgray'))

print("\nMatriz Reconstruída e Preenchida pelo SVD:")
display(reconstructed_df.style.background_gradient(cmap='viridis'))


In [ ]:
"""
### Passo 4 (Avançado): Olhando as Matrizes P e Q

Podemos inspecionar as matrizes de fatores latentes `P` (usuários) e `Q` (itens) que o SVD aprendeu. Cada linha é o "perfil" de um usuário ou item em um espaço de `k=2` dimensões.
"""
# Obtendo as matrizes P e Q do modelo treinado
P = algo.pu # Fatores dos usuários (n_users x k)
Q = algo.qi # Fatores dos itens (n_items x k)

# Mapeando os IDs internos do Surprise para os nomes originais
user_map = [trainset.to_raw_uid(inner_id) for inner_id in range(trainset.n_users)]
item_map = [trainset.to_raw_iid(inner_id) for inner_id in range(trainset.n_items)]

# Criando DataFrames para P e Q
df_P = pd.DataFrame(P, index=user_map, columns=['Fator_1', 'Fator_2'])
df_Q = pd.DataFrame(Q, index=item_map, columns=['Fator_1', 'Fator_2'])

print("Matriz P (Perfis dos Usuários):")
print(df_P)

print("\nMatriz Q (Perfis dos Itens):")
print(df_Q)

# Verificação: o produto P * Q.T deve ser próximo da matriz reconstruída
# Pegando os vetores para 'Ana' e 'Matrix'
ana_vector = df_P.loc['Ana'].values
matrix_vector = df_Q.loc['Matrix'].values

# Adicionando os "biases" (vieses) que o SVD também aprende
ana_bias = algo.bu[trainset.to_inner_uid('Ana')]
matrix_bias = algo.bi[trainset.to_inner_iid('Matrix')]
global_mean = trainset.global_mean

predicted_rating_manual = global_mean + ana_bias + matrix_bias + np.dot(ana_vector, matrix_vector)
print(f"\nPrevisão manual para (Ana, Matrix) via dot product: {predicted_rating_manual:.2f}")
print(f"Previsão do modelo para (Ana, Matrix): {reconstructed_df.loc['Matrix', 'Ana']:.2f}")


In [ ]:
"""
### Conclusão

A Fatoração de Matrizes é uma abordagem poderosa que vai além de simplesmente encontrar vizinhos. Ela aprende **representações latentes** (perfis) para usuários e itens.

**Vantagens:**
-   **Generalização:** O modelo aprende características gerais, o que o ajuda a fazer previsões mesmo para usuários ou itens com poucas avaliações.
-   **Esparsidade:** Lida muito bem com matrizes esparsas (com muitos valores faltantes), que é o caso comum em cenários reais.
-   **Escalabilidade:** É computacionalmente mais eficiente para fazer previsões do que os métodos baseados em vizinhança em larga escala.

Este método é a base de muitos sistemas de recomendação modernos e sofisticados.
"""